In [12]:
from pydantic import BaseModel, Field, validate_call
from pydantic.dataclasses import dataclass
from typing import Optional
from pathlib import Path


class GenericDirectory(BaseModel):
    """
    Generic directory with a common start that would be replaced.
    """

    generic: str = Field(description="generic part of the path to be replaced")
    replacement: Optional[Path] = Field(
        description="leave as None if the replacement path must be set by each process"
    )

    @validate_call
    def specify(self, generic_path: Path) -> Path:
        if self.replacement is None:
            raise ValueError("self.replacement cannot be None for this method")
        generic_path = Path(generic_path)
        parts = generic_path.parts
        try:
            index = parts.index(self.generic)
        except ValueError:
            raise ValueError(
                f"Path does not contain the expected generic component: '{self.generic}', in path: '{generic_path}'"
            )
        return Path(self.replacement).joinpath(*parts[index + 1 :])

    @validate_call
    def unspecify(self, specific_path: Path) -> Path:
        raise NotImplementedError(
            "Unspecifying a specific path is not supported yet, but is planned"
        )


In [8]:
generic = 'raw'
replacement = '/mnt/scratch11/Mouse'
test_path = '/raw/Ephys/test'

gd = GenericDirectory(generic=generic, replacement=replacement)
gd.specify(test_path)

WindowsPath('/mnt/scratch11/Mouse/Ephys/test')

In [16]:
### Experimenting
@dataclass
class GenericFilepathContext: # This isn't transparent Dependency Injection (DI)
    generic: str = Field(description="generic part of the path to be replaced")
    replacement: Optional[Path] = Field(
        description="leave as None if the replacement path must be set by each process"
    )

    def __enter__(self):
        print("Enter: Acquiring resource.")
        return self  # This object is bound to the target of the `with` statement

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exit: Releasing resource.")
        # Handle exception, if any, and return False to propagate or True to suppress

    
    @validate_call
    def specify(self, generic_path: Path) -> Path:
        print("Specifying")
        if self.replacement is None:
            raise ValueError("self.replacement cannot be None for this method")
        generic_path = Path(generic_path)
        parts = generic_path.parts
        try:
            index = parts.index(self.generic)
        except ValueError:
            raise ValueError(
                f"Path does not contain the expected generic component: '{self.generic}', in path: '{generic_path}'"
            )
        return Path(self.replacement).joinpath(*parts[index + 1 :])

    @validate_call
    def unspecify(self, specific_path: Path) -> Path:
        raise NotImplementedError(
            "Unspecifying a specific path is not supported yet, but is planned"
        )

# Usage
with GenericFilepathContext(generic, replacement) as resource:
    print(resource.specify(test_path))

Enter: Acquiring resource.
Specifying
\mnt\scratch11\Mouse\Ephys\test
Exit: Releasing resource.


In [ ]:
### Experimenting
@dataclass
class PipelineConfig: # This isn't transparent Dependency Injection (DI)
    config_name: str

    def __enter__(self):
        print("Enter: Acquiring resource.")
        return self  # This object is bound to the target of the `with` statement

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exit: Releasing resource.")
        # Handle exception, if any, and return False to propagate or True to suppress

# Usage
with GenericFilepathContext(generic, replacement) as resource:
    print(resource.specify(test_path))

In [17]:
from neuropixel_pipeline.schemata import ephys

In [22]:
ephys.EphysRecording.populate

Signature:
ephys.EphysRecording.populate(
    *restrictions,
    suppress_errors=False,
    return_exception_objects=False,
    reserve_jobs=False,
    order='original',
    limit=None,
    max_calls=None,
    display_progress=False,
)
Source:   
    def populate(self, *restrictions, suppress_errors=False, return_exception_objects=False,
                 reserve_jobs=False, order="original", limit=None, max_calls=None,
                 display_progress=False):
        """
        rel.populate() calls rel.make(key) for every primary key in self.key_source
        for which there is not already a tuple in rel.
        :param restrictions: a list of restrictions each restrict (rel.key_source - target.proj())
        :param suppress_errors: if True, do not terminate execution.
        :param return_exception_objects: return error objects instead of just error messages
        :param reserve_jobs: if true, reserves job to populate in asynchronous fashion
        :param order: "original"|"re